## Various ways to get Options expiration dates

In [96]:
import os,sys
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as pdr
import datetime
import pytz
import pathlib
from dateutil.relativedelta import *
import pandas_market_calendars as pmc
import pathlib
from dashapp import dashapp2 as dashapp
import os,sys
import requests
import io
import subprocess
import re
import xml.etree.ElementTree as ET
import xmltodict
import zipfile
import chrome_driver_upload as cdu
from pandas.tseries.holiday import USFederalHolidayCalendar
import sel_scrape as sc
from tqdm import tqdm,tqdm_notebook

In [27]:
DOWNLOAD_CHROMEDRIVER = False
MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_CODE = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}
TIMEZONE = 'US/Eastern'
CHROME_DOWNLOADED=False

In [28]:
if DOWNLOAD_CHROMEDRIVER and !CHROME_DOWNLOADED:
    cdu.download_chromedriver()
    CHROME_DOWNLOADED=True
sac = sc.SelScrape(headless=False,driver_name='chrome')#,profile_path=pp)
sac.goto('https://www.cmegroup.com/tools-information/quikstrike/options-calendar.html')


SyntaxError: invalid syntax (<ipython-input-28-d74b966e0f59>, line 1)

In [19]:
sac.goto("https://cmegroup-tools.quikstrike.net/User/QuikStrikeView.aspx?viewitemid=IntegratedCMEOptionExpirationCalendar&userId=undefined&jobRole=undefined&company=undefined&companyType=undefined&insid=36644309")
# sac.findxpath("//div[@class='cmeIframe section']")

2020-06-28 07:55:52,259 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:55194/session/5665c6663b0cf352cfa85d4750b4d152/url {"url": "https://cmegroup-tools.quikstrike.net/User/QuikStrikeView.aspx?viewitemid=IntegratedCMEOptionExpirationCalendar&userId=undefined&jobRole=undefined&company=undefined&companyType=undefined&insid=36644309"}
2020-06-28 07:55:53,734 - urllib3.connectionpool - DEBUG - http://127.0.0.1:55194 "POST /session/5665c6663b0cf352cfa85d4750b4d152/url HTTP/1.1" 200 14
2020-06-28 07:55:53,735 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request


In [104]:
df_eee_all = None
for i in tqdm(range(1,7)):
    url = f'https://cmegroup-tools.quikstrike.net/User/Export/CME/ExpirationCalendar.aspx?GroupId={i}'
    df_eee_temp = pd.read_csv(url)
    if df_eee_all is None:
        df_eee_all = df_eee_temp.copy()
    else:
        df_eee_all = df_eee_all.append(df_eee_temp,ignore_index=True)
    df_eee_all.index = list(range(len(df_eee_all)))

100%|██████████| 6/6 [00:05<00:00,  1.08it/s]


In [105]:
df_eee_all

,Option First Trade Date,Option Expiration Date (CT),Option Product,Option Symbol,Underlying Symbol,Underlying Expiration Date (CT)
0,06/05/2020,07/02/2020 13:20:00,Corn Weekly Options Wk 1,ZC1N0,ZCU0,09/14/2020 12:01:00
1,06/12/2020,07/10/2020 13:20:00,Corn Weekly Options Wk 2,ZC2N0,ZCU0,09/14/2020 12:01:00
2,06/19/2020,07/17/2020 13:20:00,Corn Weekly Options Wk 3,ZC3N0,ZCU0,09/14/2020 12:01:00
3,04/24/2020,07/24/2020 13:20:00,Corn Options,OZCQ0,ZCU0,09/14/2020 12:01:00
4,07/20/2018,07/24/2020 13:20:00,Short-Dated New Crop Corn Options,OCDQ0,ZCZ0,12/14/2020 12:01:00
...,...,...,...,...,...,...
3151,10/28/2020,08/25/2022 12:00:00,Copper Options,HXEU2,HGU2,09/28/2022 12:00:00
3152,11/25/2020,09/27/2022 12:00:00,Copper Options,HXEV2,HGZ2,12/28/2022 12:00:00
3153,12/29/2020,10/26/2022 12:00:00,Copper Options,HXEX2,HGZ2,12/28/2022 12:00:00
3154,01/27/2021,11/22/2022 11:00:00,Copper Options,HXEZ2,HGZ2,12/28/2022 12:00:00


In [113]:
prod_pattern = 'CL'#'[A-Z0-9][A-Z]{1,2}'
regex_pattern = '^'+prod_pattern+'['+MONTH_CODES+'][0-9]{1,2}$'
regex_pattern = '^'+prod_pattern+'['+'Q'+'][0-9]{1,2}$'

df_eee_all2 = df_eee_all[df_eee_all['Underlying Symbol'].str.contains(regex_pattern)]
display(df_eee_all2['Underlying Symbol'].str.slice(0,).unique())

array(['CLQ0', 'CLQ1', 'CLQ2', 'CLQ3', 'CLQ4', 'CLQ5', 'CLQ6', 'CLQ7',
       'CLQ8', 'CLQ9', 'CLQ30', 'CLQ31'], dtype=object)

In [114]:
df_eee_all[~df_eee_all['Option Symbol'].isna()]

,Option First Trade Date,Option Expiration Date (CT),Option Product,Option Symbol,Underlying Symbol,Underlying Expiration Date (CT)
0,06/05/2020,07/02/2020 13:20:00,Corn Weekly Options Wk 1,ZC1N0,ZCU0,09/14/2020 12:01:00
1,06/12/2020,07/10/2020 13:20:00,Corn Weekly Options Wk 2,ZC2N0,ZCU0,09/14/2020 12:01:00
2,06/19/2020,07/17/2020 13:20:00,Corn Weekly Options Wk 3,ZC3N0,ZCU0,09/14/2020 12:01:00
3,04/24/2020,07/24/2020 13:20:00,Corn Options,OZCQ0,ZCU0,09/14/2020 12:01:00
4,07/20/2018,07/24/2020 13:20:00,Short-Dated New Crop Corn Options,OCDQ0,ZCZ0,12/14/2020 12:01:00
...,...,...,...,...,...,...
3151,10/28/2020,08/25/2022 12:00:00,Copper Options,HXEU2,HGU2,09/28/2022 12:00:00
3152,11/25/2020,09/27/2022 12:00:00,Copper Options,HXEV2,HGZ2,12/28/2022 12:00:00
3153,12/29/2020,10/26/2022 12:00:00,Copper Options,HXEX2,HGZ2,12/28/2022 12:00:00
3154,01/27/2021,11/22/2022 11:00:00,Copper Options,HXEZ2,HGZ2,12/28/2022 12:00:00


In [2]:
uk_holidays = open('expiration_data/uk_holidays.csv').readlines()
# uksplit = [','.join([t.strip('\n') for t in l.split(',')]) for l in  uk_holidays]
uksplit = [','.join(l.split(',')) for l in  uk_holidays]
fio = io.StringIO()
fio.writelines(uksplit)
fio.seek(0)
df_ukh = pd.read_csv(fio)
def ukh_to_yyyymmdd(month_day,year):
    md = month_day.strip()
    d = datetime.datetime.strptime(f'{md} {year}', '%B %d %Y')
    return d
#     return d.strftime("%Y-%m-%d")

year_cols = [c for c in df_ukh.columns.values if '20' in str(c)]
for c in year_cols:
    df_ukh[c] = df_ukh[c].apply(lambda s:ukh_to_yyyymmdd(s,c))
uk_holidays = sorted(df_ukh[year_cols].values.reshape(-1))
uk_holidays

[numpy.datetime64('2020-01-02T00:00:00.000000000'),
 numpy.datetime64('2020-04-10T00:00:00.000000000'),
 numpy.datetime64('2020-04-13T00:00:00.000000000'),
 numpy.datetime64('2020-05-04T00:00:00.000000000'),
 numpy.datetime64('2020-05-25T00:00:00.000000000'),
 numpy.datetime64('2020-08-31T00:00:00.000000000'),
 numpy.datetime64('2020-11-30T00:00:00.000000000'),
 numpy.datetime64('2020-12-25T00:00:00.000000000'),
 numpy.datetime64('2020-12-28T00:00:00.000000000'),
 numpy.datetime64('2021-01-01T00:00:00.000000000'),
 numpy.datetime64('2021-04-02T00:00:00.000000000'),
 numpy.datetime64('2021-04-05T00:00:00.000000000'),
 numpy.datetime64('2021-05-03T00:00:00.000000000'),
 numpy.datetime64('2021-05-31T00:00:00.000000000'),
 numpy.datetime64('2021-08-30T00:00:00.000000000'),
 numpy.datetime64('2021-11-30T00:00:00.000000000'),
 numpy.datetime64('2021-12-27T00:00:00.000000000'),
 numpy.datetime64('2021-12-28T00:00:00.000000000'),
 numpy.datetime64('2022-01-03T00:00:00.000000000'),
 numpy.datet

In [3]:
bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())
# bday_uk = pd.offsets.CustomBusinessDay(calendar=pmc.exchange_calendar_ice.ICEExchangeCalendar().regular_holidays)
bday_uk = pd.offsets.CustomBusinessDay(holidays=uk_holidays)


In [5]:

def get_nth_weekday(year,month,target_weekday,nth_occurrence):
    '''
    weekday is the term that assigns numbers from 0 to 6 to the days of the weeks.
    weekday 0 = monday
    '''
    # get dayofweeks of year,month,1
    weekday_01 = datetime.datetime(year,month,1).weekday()
    if weekday_01 <= target_weekday:
        day_of_month_of_first_occurence = target_weekday - weekday_01
        day_of_month_of_nth_occurence = day_of_month_of_first_occurence + 1 + (nth_occurrence - 1) * 7
    else:
        day_of_month_of_nth_occurence = target_weekday - weekday_01 + 1 + (nth_occurrence) * 7 
    return datetime.datetime(year,month,day_of_month_of_nth_occurence)




def get_ES_expiry(symbol):
    '''
    3rd friday of month of symbol
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return get_nth_weekday(year,month,4,3)

def get_E6_expiry(symbol):
    monthcode_yy = symbol[2:]
    next_month = DICT_MONTH_CODE[monthcode_yy[0]] + 1
    year = 2000 + int(monthcode_yy[1:])
    if next_month>12:
        next_month = 1
        year += 1
    return datetime.datetime(year,next_month,1) - 7*bday_us

def get_CL_expiry(symbol):
    '''
    Trading terminates 7 business days before the 26th calendar of the month prior to the contract month.
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month -1
    if month<1:
        month = 12
        year = year - 1
    return datetime.datetime(year,month,26) - 7*bday_us

def get_NG_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return datetime.datetime(year,month,1) - 4*bday_us

def get_CB_expiry(symbol):
    '''
    This is the spec for the CME Brent, but it matches ICE.
    Trading terminates the 4th last London business day of 
    the month, 2 months prior to the contract month 
    except for the February contract month which 
    terminates the 5th last London business day of the 
    month, 2 months prior to the contract month.  
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month - 1
    if month<1:
        month = 12 + month
        year = year - 1
    days_to_subtract = 4
    if monthcode_yy[0] =='G':
        days_to_subtract = 5
    elif monthcode_yy[0] == 'F':
        days_to_subtract = 3
#     elif monthcode_yy == 'N22':
#         days_to_subtract = 7
    return datetime.datetime(year,month,1,0,0) - days_to_subtract * bday_uk

DICT_PRODUCT = {
    'E6':get_E6_expiry,
    'ES':get_ES_expiry,
    'CL':get_CL_expiry,
    'NG':get_NG_expiry,
    'CB':get_CB_expiry,
}

    
def get_expiry(symbol):
    product = symbol[:2]
    f = DICT_PRODUCT[product]
    return f(symbol)


def dt_from_yyyymmdd(yyyymmdd,hour=0,minute=0,timezone=TIMEZONE):
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])  
    return datetime.datetime(y,m,d,hour,minute,tzinfo=pytz.timezone(timezone))

def yyyymmdd_from_dt(dt):
    y = int(dt.year)
    m = int(dt.month)
    d = int(dt.day)
    return y*100*100 + m*100 + d

def get_dte_pct(trade_yyyymmdd,expiry_yyyymmdd):
    dt_td = dt_from_yyyymmdd(trade_yyyymmdd)
    dt_xp = dt_from_yyyymmdd(expiry_yyyymmdd)
    return ((dt_xp - dt_td).days + 1)/365


In [6]:
# get options expirations from barchart using curl
# c = 'CB'
# ybeg = 20
# yend = 23
# for y in range(ybeg,yend+1):
#     !bash barchart_expirations.sh {c} {y}

In [271]:
# get energy expirations from the cme group
cme_expiry_url="https://cmegroup-tools.quikstrike.net/CME/Export/Expirations.aspx?insid=36580046"
with requests.Session() as s:
    download = s.get(cme_expiry_url)
    decoded_content = download.content.decode('utf-8')
s = io.StringIO(decoded_content)
lines = s.readlines()

# cme_expiry_url=f"{pathlib.Path.home()}/downloads/Expirations.csv"
# lines = open(cme_expiry_url,'r').readlines()

lines2 = [l.strip().split(',') for l in lines]
lfix = lambda l: l[0:1] + [l[1] + ' ' + l[2]] + l[3:]
lines2 = [l if len(l)==15 else lfix(l) for l in lines2]
# df_current_expirations = pd.read_csv(cme_expiry_url)
df_cme_expirations = pd.DataFrame(lines2[1:],columns=lines2[0])

df_expiry = df_cme_expirations.copy()
df_expiry['symbol'] = df_expiry.Symbol
df_expiry['expiry'] = [datetime.datetime.strptime(str_date,'%m/%d/%Y %H:%M') for str_date in df_expiry.Expiration]
syms_search = df_expiry.Symbol.str.contains('[FGHJKMNQUVXZ][0-9]$',regex=True)
syms = df_expiry.loc[syms_search,'Symbol'].values
sym_ys = [int(s[-1]) for s in syms ]
expiries = df_expiry.loc[syms_search,'Expiration'].values
expiries_yys = [int(expiry[6:10])-2000 for expiry in expiries]
this_decade = int(str(int(datetime.datetime.now().year))[2])*10
symbol_decades = [this_decade + (10 if this_decade<expiries_yys[i] else 0) for i in range(len(syms))]
new_syms = [syms[i][:-1] + str(symbol_decades[i]+sym_ys[i]) for i in range(len(syms))]
df_expiry.loc[syms_search,'symbol'] = new_syms
df_expiry[df_expiry.Symbol.str.contains('^LO[A-Z][0-9]$',regex=True)].sort_values('expiry')

len(df_expiry)


2020-06-26 11:27:38,619 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): cmegroup-tools.quikstrike.net:443
2020-06-26 11:27:42,619 - urllib3.connectionpool - DEBUG - https://cmegroup-tools.quikstrike.net:443 "GET /CME/Export/Expirations.aspx?insid=36580046 HTTP/1.1" 200 14114284


98481

In [300]:
# get energy expirations from the cme group
def create_energy_expiry():
    cme_expiry_url="https://cmegroup-tools.quikstrike.net/CME/Export/Expirations.aspx?insid=36580046"
    with requests.Session() as s:
        download = s.get(cme_expiry_url)
        decoded_content = download.content.decode('utf-8')
    s = io.StringIO(decoded_content)
    lines = s.readlines()

    # cme_expiry_url=f"{pathlib.Path.home()}/downloads/Expirations.csv"
    # lines = open(cme_expiry_url,'r').readlines()

    lines2 = [l.strip().split(',') for l in lines]
    lfix = lambda l: l[0:1] + [l[1] + ' ' + l[2]] + l[3:]
    lines2 = [l if len(l)==15 else lfix(l) for l in lines2]
    # df_current_expirations = pd.read_csv(cme_expiry_url)
    df_cme_expirations = pd.DataFrame(lines2[1:],columns=lines2[0])

    df_expiry = df_cme_expirations.copy()
    df_expiry['symbol'] = df_expiry.Symbol
    df_expiry['expiry'] = [datetime.datetime.strptime(str_date,'%m/%d/%Y %H:%M') for str_date in df_expiry.Expiration]
    syms_search = df_expiry.Symbol.str.contains('[FGHJKMNQUVXZ][0-9]$',regex=True)
    syms = df_expiry.loc[syms_search,'Symbol'].values
    sym_ys = [int(s[-1]) for s in syms ]
    expiries = df_expiry.loc[syms_search,'Expiration'].values
    expiries_yys = [int(expiry[6:10])-2000 for expiry in expiries]
    this_decade = int(str(int(datetime.datetime.now().year))[2])*10
    symbol_decades = [this_decade + (10 if this_decade<expiries_yys[i] else 0) for i in range(len(syms))]
    new_syms = [syms[i][:-1] + str(symbol_decades[i]+sym_ys[i]) for i in range(len(syms))]
    df_expiry.loc[syms_search,'symbol'] = new_syms
    df_expiry[df_expiry.Symbol.str.contains('^LO[A-Z][0-9]$',regex=True)].sort_values('expiry')
    return df_expiry



In [38]:
get_platform_architecture()

('mac', '64')

In [51]:
!ls {'./drivers'}

In [328]:
vers = r[0]
# tree.findall(f"//key[contains(text(),'{vers}')]")
for el in tree.findall('.'):
    print(el)

<Element {http://doc.s3.amazonaws.com/2006-03-01}ListBucketResult at 0x15820ba48>


In [274]:
df_expiry.to_csv('cme_energy_expirations.csv',index=False)

In [284]:
df_expiry.columns.values

array(['Product Group', 'Product', 'Symbol', 'Underlying', 'First Trade',
       'Expiration', 'Settle', 'Clearing', 'Globex', 'Prs', 'Floor',
       'Group', 'Itc', 'Exchange', 'Contract Type', 'symbol', 'expiry'],
      dtype=object)

In [298]:
df_lo = df_expiry[df_expiry.symbol.str.contains('^LO[FGHJKMNQUVXZ][2-4][0-9]')][['Symbol','Underlying','expiry','symbol']]
clsyms = [f'CL{s}' for s in [l[2:] for l in df_lo.symbol]]
theo_expiries = [get_expiry(s) for s in clsyms]
df_lo['theo'] = theo_expiries
df_lo['actual'] = [datetime.datetime(ex.year,ex.month,ex.day) for ex in df_lo.expiry]
df_lo['theo_vs_actual'] = df_lo.actual == df_lo.theo
df_lo[~df_lo.theo_vs_actual][['Symbol','Underlying','symbol','actual','theo']]


,Symbol,Underlying,symbol,actual,theo
13949,LOG1,CLG1,LOG31,2021-01-14,2031-01-15
13950,LOH1,CLH1,LOH31,2021-02-17,2031-02-14
13951,LOJ1,CLJ1,LOJ31,2021-03-17,2031-03-17
13952,LOK1,CLK1,LOK31,2021-04-15,2031-04-17
13953,LOM1,CLM1,LOM31,2021-05-17,2031-05-15
...,...,...,...,...,...
14052,LOU9,CLU9,LOU39,2029-08-16,2039-08-17
14053,LOV9,CLV9,LOV39,2029-09-17,2039-09-15
14054,LOX9,CLX9,LOX39,2029-10-17,2039-10-17
14055,LOZ9,CLZ9,LOZ39,2029-11-14,2039-11-16


In [8]:
c = "CL"
ybeg = 20
yend = 23
df_exp = None
for y in range(ybeg,yend+1):
    dft = pd.read_csv(f'expiration_data/commod_expirations_{c}{y}.csv')
    if df_exp is None:
        df_exp = dft.copy()
    else:
        df_exp = df_exp.append(dft,)
def mmddyy_to_yyyymmdd(mmddyy):
    if str(mmddyy).lower() == 'nan':
        return 99999999
    s = mmddyy.split('/')
    y =  2000 + int(s[2])
    m = int(s[0])
    d = int(s[1])
    return (y*100*100 + m*100 + d)
def yyyymmdd_from_symbol(s):
    d = get_expiry(s)
    return int(d.year*100*100 + d.month*100 + d.day)
df_exp.expiry = df_exp.expiry.apply(lambda mmddyy:mmddyy_to_yyyymmdd(mmddyy))
df_exp['theo_expiry'] = df_exp.symbol.apply(lambda s:yyyymmdd_from_symbol(s))
df_exp['theo_is_good'] = df_exp.apply(lambda r: False if (r.expiry==99999999) or (r.expiry != r.theo_expiry) else True,axis=1)
df_exp = df_exp[df_exp.expiry!=99999999]
df_exp

,symbol,expiry,theo_expiry,theo_is_good
3,CLJ20,20200317,20200317,True
4,CLK20,20200416,20200416,True
5,CLM20,20200514,20200514,True
6,CLN20,20200617,20200617,True
7,CLQ20,20200716,20200716,True
8,CLU20,20200817,20200817,True
9,CLV20,20200917,20200917,True
10,CLX20,20201015,20201015,True
11,CLZ20,20201117,20201117,True
0,CLF21,20201216,20201216,True
